In [1]:
import pandas as pd
import numpy as np

In [4]:
df=pd.read_csv("marketing_campaign_data_messy.csv")

## Step 1: Header Cleaning

In [ ]:
# df.columns.to_list()
# We saw column has white spaces
df.columns=df.columns.str.strip().str.lower().str.replace(' ','_')

## Step 2: Type Conversion & Currency Cleaning

In [ ]:
# df['spend'].str.contains(r'\$')
# df[df['spend'].str.contains(r'\$')]

# df['spend']=df['spend'].astype('str').str.replace(r'[^\d.-]','',regex=True)

# converting back to numeric
df['spend']=pd.to_numeric(df['spend'])
# df['spend']=df['spend'].astype('float')

dtype('float64')

## Step 3: Categorical Typos (Fuzzy Logic)

In [ ]:
# df['channel'].unique()

# df['channel']=df['channel'].replace({'E-mail':'Email','Gogle':'Google Ads','Facebok':'Facebook','Insta_gram':'Instagram','Tik_Tok':'TikTok','nan':np.nan})

# or

channel_map={
    'E-mail':'Email',
    'Gogle':'Google Ads',
    'Facebok':'Facebook',
    'Insta_gram':'Instagram',
    'Tik_Tok':'TikTok',
    'nan':np.nan
}
df['channel']=df['channel'].replace(channel_map)
df['channel'].unique()

array(['TikTok', 'Facebook', 'Email', 'Instagram', 'Google Ads', nan],
      dtype=object)

## Step 4: Handling Mixed Boolean 

In [ ]:
active_map={
    'Yes':True,
    'Y':True,
    1:True,
    '1':True,
    'True':True,
    'False':False,
    'No':False,
    0:False,
    '0':False
}
df['active']=df['active'].replace(active_map)



np.int64(0)

## Step 5: Date Parsing

In [ ]:
# since dates are in object format we need to convert them into date format
df['start_date']=pd.to_datetime(df['start_date'],errors='coerce')
df['end_date']=pd.to_datetime(df['end_date'],dayfirst=True,errors='coerce')


## Step 6: Logical Integrity Click and Impression

In [67]:
# since we have two click column we need to drop the duplicated
# df=df.loc[:,~df.columns.duplicated()]

# we perform this to check because impressions should always be greater than clicks
check_mask=df['clicks']>df['impressions']
df.loc[check_mask]

# or
df.loc[df['clicks']>df['impressions']]

,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag


## Step 7: Logical Time Integrity

In [ ]:
# here we are checking if end_date is in past of start_date
time_mask=df['end_date']<df['start_date']
# df.loc[time_mask]

# here i add 30 days to end date from start date , since usually campaign last 30 days

df.loc[time_mask,'end_date']=df.loc[time_mask,'start_date']+pd.Timedelta(days=30)

df.loc[time_mask,['campaign_id','start_date','end_date']]

,campaign_id,start_date,end_date
23,CMP-00024,2023-05-06,2023-06-05
54,CMP-00055,2023-09-01,2023-10-01
97,CMP-00098,2023-07-02,2023-08-01
124,CMP-00125,2023-04-01,2023-05-01
218,CMP-00219,2023-10-03,2023-11-02
258,CMP-00259,2023-05-03,2023-06-02
344,CMP-00345,2023-12-04,2024-01-03
346,CMP-00347,2023-11-07,2023-12-07
380,CMP-00381,2023-05-01,2023-05-31
386,CMP-00387,2023-11-01,2023-12-01


## Step 8: String Parsing - Extracting Campaign

In [ ]:
df['campaign_name'].head()
df['season']=df['campaign_name'].str.extract(r'Q\d_([^_]+)_')


,campaign_name,season
0,False,NaN
1,Q1_Launch_CMP-00002,Launch
2,Q3_Winter_CMP-00003,Winter
3,False,NaN
4,False,NaN
...,...,...
2015,False,NaN
2016,Q4_Summer_CMP-01255,Summer
2017,Q2_Launch_CMP-01050,Launch
2018,False,NaN


In [ ]:
# df[['campaign_name','season']].head()

# df['season'].str.replace({'Nan':'Unknown'})

# df['campaign_name'].astype('str').str.replace('False','Unknown')

# campaign_false=df['campaign_name']==False
df.loc[df['campaign_name']!=False,:]


,campaign_id,campaign_name,start_date,end_date,channel,impressions,clicks,spend,conversions,active,campaign_tag,season
1,CMP-00002,Q1_Launch_CMP-00002,2023-05-06,2023-05-12,Facebook,1860,30,24.33,1.0,False,FA,Launch
2,CMP-00003,Q3_Winter_CMP-00003,2023-12-13,2023-12-20,Email,77820,843,1323.39,51.0,False,EM,Winter
9,CMP-00010,Q2_Winter_CMP-00010,2023-03-21,2023-04-01,Email,61263,1153,1623.56,NaN,False,EM,Winter
11,CMP-00012,Q4_Launch_CMP-00012,2023-02-19,2023-02-20,Google Ads,42090,763,1267.69,109.0,False,GO,Launch
12,CMP-00013,Q2_Launch_CMP-00013,2023-05-16,2023-06-01,Google Ads,68221,2975,5292.16,567.0,False,GO,Launch
...,...,...,...,...,...,...,...,...,...,...,...,...
2004,CMP-01602,Q1_Winter_CMP-01602,2023-11-11,2023-11-21,Google Ads,11816,374,460.42,67.0,False,GO,Winter
2012,CMP-01200,Q3_BlackFriday_CMP-01200,2023-09-25,2023-10-15,Email,54028,2254,1991.67,139.0,False,EM,BlackFriday
2014,CMP-00185,Q3_Launch_CMP-00185,2023-08-03,2023-08-27,Facebook,56016,1728,1890.33,128.0,False,FA,Launch
2016,CMP-01255,Q4_Summer_CMP-01255,2023-09-01,2023-09-26,Google Ads,20097,897,1641.0,162.0,False,GO,Summer
